#### 姓名：王子陈 

#### 学号：20171050008 

#### 专业：电子科学与技术

## 一维势阱电势分布

势阱底面电势为1,其余三面电势为0，求势阱内电势分布。

![](./仿真-一维势阱.png)

因为阱内无电荷，所以求解拉普拉斯方程：

$$
\nabla^2\phi=0 \\
\frac{\phi(x+\Delta x,y)+\phi(x-\Delta x,y)-2\phi(x,y)}{\Delta x^2}+
\frac{\phi(x,y+\Delta y)+\phi(x,y-\Delta y)-2\phi(x,y)}{\Delta y^2}=0 \\
[\phi(x+\Delta x,y)+\phi(x-\Delta x,y)-2\phi(x,y)]\Delta y^2=
[-\phi(x,y+\Delta y)-\phi(x,y-\Delta y)+2\phi(x,y)]\Delta x^2 \\
\phi(x,y)=
\frac{[\phi(x+\Delta x,y)+\phi(x-\Delta x,y)]\Delta y^2+[\phi(x,y+\Delta y)+\phi(x,y-\Delta y)]\Delta x^2}
{2(\Delta x^2+\Delta y^2)}
$$

### 源代码：

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xlength=1e-3  #x方向长度 1mm （单位 m）
ylength=1e-3  #y方向长度 0.1mm
n = 10       #100行
m = 10       #100列
phiArray=np.zeros((n,m))

phiArray[:,0]=1  #矩阵第一列置1

for i in range(10000):
    phiArray[1:-1,1:-1]=((phiArray[1:-1,2:]+phiArray[1:-1,0:-2])*((xlength/n)**2) \
                     +(phiArray[2:,1:-1]+phiArray[0:-2,1:-1])*((ylength/m)**2)) \
                     /(2*((xlength/n)**2+(ylength/m)**2))
    
zeroArray=np.zeros((n,1))
for i in range(n):
    if( list(phiArray[:,i])==list(zeroArray[:,0])):
        print(i)
        break
print(phiArray[:,m-2])
    
plt.pcolor(phiArray.T)    #pcolor绘制的顺序是反的
plt.colorbar()
plt.xlabel("x",fontsize=24)
plt.ylabel("y",fontsize=24)
plt.savefig('./1x1mm_10x10_1wTimes')
plt.show()

改变 x 方向和 y 方向的长度以及 dx 和 dy (也就是 n 和 m)的值,计算、做图并分析这些参数对最终电势分布的影响。

以下图像都迭代了一万次。

### 参照组
x长度=1mm, y长度=1mm, n=100格, m=100格

dx=10$\mu$m, dy=10$\mu$m

观察尺度比较小，x,y的观察范围都是1mm，每一格表示长度为10微米。

![](./1x1mm_100x100_1wTimes.png)


### y方向长度增加
x长度=1mm, y长度=1cm, n=100格, m=100格

dx=10$\mu$m, dy=0.1mm。

y方向一格代表0.1毫米，跨度比较大，超过1毫米就看不出色彩变化了，其实还有很微弱的数值：第98行(1cm处)的数值量级在$10^{-25}-10^{-23}$

![](./1x10mm_100x100_1wTimes.png)


### y方向长度减小
x长度=1mm, y长度=0.1mm, n=100格， m=100格

dx=10$\mu$m，dy=100$\mu$m

y方向观测范围为0-0.1mm，在此范围内电势比较大

![](./1x0.1mm_100x100_1wTimes.png)


### 改变x方向长度变化不大

在x方向上，电势是均匀的，只是拉长了

下图是x长度为1厘米的图像：

![](./10x1mm_100x100_1wTimes.png)


### m(行数)减少
x长度=1mm, y长度=1mm, n=100格, m=10格

dx=10$\mu$m，dy=0.1mm

y方向上1格长度为0.1毫米，看起来没那么细腻，y方向相邻行有明显突变

![](./1x1mm_100x10_1wTimes.png)


### m(行数)增多
x长度=1mm, y长度=1mm, n=100格, m=1000格

dx=10$\mu$m, dy=1$\mu$m

y方向1格长度为1微米，没有颗粒感，更细腻

![](./1x1mm_100x1000_1wTimes.png)


### n(列数)减少
x长度=1mm, y长度=1mm, n=10格, m=100格

x方向1格长度100$\mu$m，x方向有明显突变

![](./1x1mm_10x100_1wTimes.png)

### m, n都减少
x长度=1mm, y长度=1mm, n=10格, m=10格

x方向和y方向1格都是距离0.1毫米，颗粒感比较强
![](./1x1mm_10x10_1wTimes.png)

## 针尖放电 Point Discharge

盒子底面放一个针尖等势体，三面电势为0。（底边和红色区域电势为固定值）

![](./仿真-针尖.png)

针尖：两条直线拼起来

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#%matplotlib notebook

xLength=1e-3   #x方向长度 1mm （单位 m）
yLength=1e-3 #y方向长度 0.1mm
n = 100        #矩阵100行
m = 100        #矩阵100列
dx = xLength/n #x方向一格长度
dy = yLength/m #y方向一格长度
phiArray=np.zeros((n,m))
volt = 0.1       #电势

height = yLength*0.5    #针尖高度
width = xLength*0.1   #1/2针尖宽度
slop_1 = height/width #直线1斜率
slop_2 = -slop_1
x1 = xLength/2-width  #直线1截距 x1到原点距离
x2 = xLength/2+width  #直线2截距 x2到原点距离
x1Num = x1/dx  #截距x1所在网格序号
x2Num = x2/dx  #截距x2所在网格序号

# 初始化等势体电势
def initialize_point(phiArray, slop_1, slop_2, x1Num, x2Num, volt):
    phiArray[:,0]=volt
    for i in range(n):
        for j in range(m):
            if ((j*dy/dx)<=slop_1*(i-x1Num)) \
            and ((j*dy/dx)<=slop_2*(i-x2Num)): 
                phiArray[i][j]=volt
# 迭代
initialize_point(phiArray, slop_1, slop_2, x1Num, x2Num, volt)
for i in range(100):
    phiArray[1:-1,1:-1]=((phiArray[1:-1,2:]+phiArray[1:-1,:-2])*((xLength/n)**2) \
                     +(phiArray[2:,1:-1]+phiArray[:-2,1:-1])*((yLength/m)**2)) \
                     /(2*((xLength/n)**2+(yLength/m)**2)) #yLength<1时，yLength/m和xLength/n要换下位置
    initialize_point(phiArray, slop_1, slop_2, x1Num, x2Num, volt) #等势体的电势每次迭代不能变
# 计算电场
Ex = -(phiArray[2:,1:-1]-phiArray[:-2,1:-1])/(2*dx)  #电势在x方向梯度的相反数
Ey = -(phiArray[1:-1,2:]-phiArray[1:-1,:-2])/(2*dy)  #电势在y方向梯度的相反数

plt.rcParams['figure.figsize'] = (15.0, 5.0)
plt.subplot(1,2,1)
plt.pcolor(phiArray.T)
plt.colorbar()
plt.xlabel("x", fontsize = 25)
plt.ylabel("y", fontsize = 25)
plt.title("电势分布", fontsize = 25)

plt.subplot(1,2,2)
plt.pcolor((Ex**2+Ey**2)[round(0.2*n):round(0.7*n),:].T)
plt.colorbar()
plt.xlabel("x", fontsize = 25)
plt.ylabel("y", fontsize = 25)
plt.title("电场强度", fontsize = 25)
plt.savefig("./p1x1_100x100_0.5x0.2_v=0.1.png")  #命名的数字单位是mm
print("max E=",np.max((Ex**2+Ey**2)[round(0.2*n):round(0.8*n),:]))

改变 x 方向和 y 方向的长度、dx 和 dy(也就是 n 和 m)的值、针尖的高度和宽度、电势大小等数据,计算、做图并分析这些参数对最终电势分布的影响。

### 参照组
x长度=1mm, y长度=1mm, n=100, m=100, height=0.5mm, width=0.2mm

容器x方向长1毫米，y方向高1毫米，针高度为容器高度的一半：0.5毫米，针的1/2宽度为容器宽的0.1倍：0.1毫米，所以针宽0.2毫米

x方向一个网格长dx=10$\mu$m, y方向一个网格长dy=10$\mu$m。电势分布及电场强度分布如下，针尖处具有最大场强为：

$max E= 6.139\times 10^{8}$
![](./p1x1_100x100_0.5x0.2.png)

### 容器y方向加长到10毫米
x方向长度=1mm， y方向长度=10mm， n=100, m=100， dx=0.01mm， dy=0.1mm， height=5mm， width=0.2mm

因为场强等于电势的微分：
$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\Delta x})^2+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\Delta y\ \uparrow})^2\downarrow
$$
dy变大，场强变小。虽然分子上的变化率也变大了，但分母(把$\phi$也差分)是三次的增速更快，整体变小。

针高远大于针宽，像一条竖线，可能针尖端y方向的变化太小而忽略，类似于截顶

$max E=1.144\times 10^{8}$
![](./p1x10_100x100_5x0.2.png)


### 容器y方向缩短到0.1毫米
xLength=1mm， yLength=0.1mm，n=100， m=100， dx=0.01mm， dy=0.001 m， height=0.05mm， width=0.2mm
$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\Delta x})^2+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\Delta y\ \downarrow})^2\uparrow
$$
dy变小，场强变大。

$max\ E=1.065\times 10^{10}$
![](./p1x0.1_100x100_0.05x0.2.png)

### 容器x方向加长到10毫米
xLength=10mm， yLength=1mm， n=100， m=100， dx=0.1mm， dy=0.01mm， height=0.5mm， width=2mm.
$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\Delta x\ \uparrow})^2\downarrow+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\Delta y})^2
$$
dx变大，场强变小。

$max\ E=1.065\times 10^{8}$

![](p10x1_100x100_0.5x2.png)

### 容器x方向减小到0.1毫米
xLength=0.1mm， yLength=1mm，n=100， m=100， dx=0.001mm， dy=0.01m， height=0.5mm， width=0.02mm
$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\Delta x\ \downarrow})^2\uparrow+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\Delta y})^2
$$
dx变小，场强变大。

$max\ E=1.144\times 10^{10}$

![](./p0.1x1_100x100_0.5x0.02.png)



### 增加针尖相对宽度
xLength=1mm， yLength=1mm， n=100， m=100， dx=0.01mm， dy=0.01mm， height=0.5mm， width=1mm

$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\Delta x}\downarrow)^2+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\Delta y}\downarrow)^2
$$
分母没变，看分子：针尖附近x方向,y方向电势变化缓慢，所以针尖尖端场强变小。

针尖的最大场强=$max E= 4.033\times 10^{8}$

![](./p1x1_100x100_0.5x1.png)

### 减小针尖相对宽度
xLength=1mm， yLength=1mm， n=100， m=100， dx=0.01mm， dy=0.01mm， height=0.5mm， width=0.1mm

容器x方向长度=1毫米，1/2针宽是容器宽度的0.05倍，所以针宽0.1毫米。

针尖高度不变，宽度减小，针尖附近x方向，y方向电势变化加剧，所以针尖尖端强度变大。

$max E=6.331\times 10^{8}$

![](./p1x1_100x100_0.5x0.1.png)

### 增加针尖相对高度
xLength=1mm， yLength=1mm， n=100， m=100， dx=0.01mm， dy=0.01mm， height=0.7mm， width=0.1mm

容器y方向高度=1毫米，针尖高度等于容器高度的0.7倍,即0.7毫米。

针尖宽度不变，高度加长，相较于参照组针尖变尖锐了，针尖两侧电势变化加剧，针尖场强变大。

$max E=6.264\times 10^{8}$

![](./p1x1_100x100_0.7x0.2.png)

### 减小针尖相对高度
xLength=1mm， yLength=1mm， n=100， m=100， dx=0.01mm， dy=0.01mm， height=0.7mm， width=0.1m

容器y方向高度=1毫米，针尖高度等于容器高度的0.3倍,即0.3毫米，针尖宽度还是0.2毫米。

针尖宽度不变，高度缩短，相较于参照组针尖变钝了，针尖两侧电势变化缓慢，针尖场强变小。

$max E = 5.772\times 10^{8}$

![](./p1x1_100x100_0.3x0.2.png)

### 增加n(列)=200
xLength=1mm， yLength=1mm， n=200， m=100， dx=0.005mm， dy=0.01mm， height=0.5mm， width=0.2m
$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\frac{xLength}{n\ \uparrow} \downarrow}\uparrow)^2+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\frac{yLength}{m}})^2
$$

n 增大，dx减小，Ex增大，场强变大：$max\ E=1.219\times 10^9$

![](./p1x1_200x100_0.5x0.2.png)



### 减小n(列)=10
xLength=1mm， yLength=1mm， n=10， m=100， dx=0.1mm， dy=0.01mm， height=0.5mm， width=0.2m
$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\frac{xLength}{n\ \downarrow} \uparrow}\downarrow)^2+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\frac{yLength}{m}})^2
$$

n 减小，dx增大，Ex减小，场强变小：$max\ E=1.410\times 10^8$

![](./p1x1_10x100_0.5x0.2.png)

### 增加m(行)=200
xLength=1mm， yLength=1mm， n=100， m=200， dx=0.01mm， dy=0.005mm， height=0.5mm， width=0.2m
$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\frac{xLength}{n} })^2+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\frac{yLength}{m\ \uparrow}\downarrow}\uparrow)^2
$$

m 增大，dy减小，Ey增大，场强变大：
$max\ E=1.705\times 10^9$

![](./p1x1_100x200_0.5x0.2.png)



### 减小m(行)=10
xLength=1mm， yLength=1mm， n=100， m=10， dx=0.01mm， dy=0.1mm， height=0.5mm， width=0.2m
$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\frac{xLength}{n} })^2+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\frac{yLength}{m\ \downarrow}\uparrow}\downarrow)^2
$$

m 减小，dy增大，Ey减小，场强变小：
$max\ E=1.037\times 10^8$

![](./p1x1_100x10_0.5x0.2.png)

### 网格10x10
xLength=1mm， yLength=1mm， n=10， m=10， dx=0.1mm， dy=0.1mm， height=0.5mm， width=0.2m
$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\frac{xLength}{n\ \downarrow}\uparrow}\downarrow)^2+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\frac{yLength}{m\ \downarrow}\uparrow}\downarrow)^2
$$

n,m 减小，dx,dy增大，Ex,Ey减小，场强变小：
$max\ E=2.479\times 10^7$

![](./p1x1_10x10_0.5x0.2.png)


### 网格200x200
x长度=1mm， y长度=1mm， n=200， m=200，dx=0.005， dy=0.005， height=0.5mm， width=0.2mm
$$
E=(\frac{\phi(x+\Delta x)-\phi(x-\Delta x)}{2\frac{xLength}{n\ \uparrow}\downarrow}\uparrow)^2+(\frac{\phi(y+\Delta y)-\phi(y-\Delta y)}{2\frac{yLength}{m\ \uparrow}\downarrow}\uparrow)^2
$$
n，m 增大，dx,dy减小，Ex,Ey增大，场强变大：

电场最强在针尖处，最大值为 $2.456\times 10^{9}$

![](./p1x1_200x200_0.5x0.2.png)


### 增大电势
等势体电势为10V，空间电势变大，针尖电势增大：$max\ E=6.139\times 10^{10}$（增大2个数量级）

![](./p1x1_100x100_0.5x0.2_v=10.png)

### 减小电势
等势体电势为0.1V，空间电势变小，针尖电势减小：$max\ E=6.139\times 10^{6}$（减小2个数量级）

![](./p1x1_100x100_0.5x0.2_v=0.1.png)